In [25]:
import numpy as np
np.random.seed(42)

# Имплементации бутстрепа

## Ванильный бутстреп

Классический бутстреп. Сгенерируем 1000 выборок с возвращением из исходной выборки и посчитаем среднее значение в каждой выборке. На выходе получаем 1000 средних значений, на основе которых можно построить доверительный интервал.

In [26]:
def loop_bootstrap(a: np.ndarray, bs_iters: int = 10000, agg="mean", **kwargs) -> np.ndarray:
    """Return a sampling with replacement with given number of iterations

    Arguments:
    a:          Dataset to sample from
    bs_iters:   Number of sampl ing iteration
    agg:        Aggregation method to use, must be in ["mean", "median", "quantile"]
    Returns:
    np.ndarray with samples of size(bs_iters, len(a))
    """
    if agg not in ["mean", "median", "quantile"]:
        raise ValueError("agg should be in ['mean', 'median', 'quantile']")
    res = []
    for _ in range(bs_iters):
        if agg == "mean":
            res.append(np.mean(np.random.choice(a, len(a), replace=True)))
        elif agg == "median":
            res.append(np.median(np.random.choice(a, len(a), replace=True)))
        elif agg == "quantile":
            res.append(np.quantile(np.random.choice(a, len(a), replace=True)), q=kwargs["q"])
    return np.array(res)




## Numpy-бутстреп

Имплементация бутстрепа через библиотеку numpy. В отличие от предыдущего решения, здесь не используется цикл, что иногда ускоряет процесс, но зато потребляет больше памяти.

In [27]:
def np_bootstrap(a: np.ndarray, bs_iters: int = 10000, agg='mean', **kwargs) -> np.ndarray:
    """Return a sampling with replacement with given number of iterations

    Arguments:
    a:          Dataset to sample from
    bs_iters:    Number of sampl ing iteration
    Returns:
    np.ndarray with samples of size(bs_iters, len(a))
    """
    samples = np.random.choice(a, (bs_iters, len(a)), replace=True)
    if agg == 'mean':
        res = np.mean(samples, axis=1)
    elif agg == 'median':
        res = np.median(samples, axis=1)
    elif agg == 'quantile':
        res = np.quantile(samples, q=kwargs['q'], axis=1)
    else:
        raise ValueError("agg should be in ['mean', 'median', 'quantile']")
    return res

## SciPy-бутстреп

Готовая имплементация бутстрепа из библиотеки scipy. Ссылка на документацию: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html

В этой имплементации много удобного функционала - можно сразу считать разницу между двумя выборками (и при необходимости бутстрапить их попарно), строить ДИ по 3м разным методам и многое другое. Рекомендую покопаться.

In [28]:
from scipy.stats import bootstrap as sp_bootstrap

## Сравнение

In [29]:
dataset = np.random.normal(10, 2, 100_000)
print('SHAPE', dataset.shape)
print('MEAN', np.mean(dataset))

SHAPE (100000,)
MEAN 10.001933736281899


In [30]:
def ci(diffs: np.ndarray, alpha: float = 0.05, **kwargs):
    # -> tuple[tuple[float, float], bool]:
    """Return a sampling with replacement with given number of iterations

    Arguments:
        diffs:  Dataset to compute percentiles upon
        alpha:  Confidence interval alpha
    Returns:
        tuple[CI left, CI right]
    """
    ci_l = np.percentile(diffs, (alpha / 2) * 100)
    ci_r = np.percentile(diffs, (1 - alpha / 2) * 100)
    return (ci_l, ci_r)

In [31]:
%%timeit
loop_bs = loop_bootstrap(dataset, bs_iters=1_000, agg='mean')

1.03 s ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
print(ci(loop_bs))

(9.980653991484209, 10.00533323276719)


In [33]:
%%timeit
np_bs = np_bootstrap(dataset, bs_iters=1_000, agg='mean')

1.46 s ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
print(ci(np_bs))

(9.979202438984808, 10.004921295735473)


In [35]:
%%timeit
sp_bs = sp_bootstrap((dataset,), statistic=np.mean, n_resamples=1_000, method='percentile')

1.5 s ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
print(sp_bs.confidence_interval)

ConfidenceInterval(low=9.979809384157058, high=10.004748508664337)
